In [ ]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as k
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

(x_training,y_training), (x_testing,y_testing) = cifar10.load_data()

In [ ]:
Batch_Size = 32
Num_of_Iterations = 200
Num_of_Classes = np.unique(y_training).shape[0]
Num_of_Channels = x_training.shape[3]
Depth = Num_of_Channels*6 + 2

In [ ]:
def Learning_Rate_Schedule(Iteration):
  Learning_rate = 1e-03
  if Iteration > 180:
    Learning_rate *= 0.5e-03
  elif Iteration > 160:
    Learning_rate *= 1e-03
  elif Iteration > 120:
    Learning_rate *= 1e-02
  elif Iteration > 80:
    Learning_rate *= 1e-01
  return Learning_rate


In [ ]:
def single_resnet_layer(inputs,Num_of_Filters=16,kernel_size=3,strides=1,activation='relu',batch_normalization=True,convol_first=True):
  convolution = Conv2D(Num_of_Filters,kernel_size=kernel_size,strides=strides,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-04))
  x=inputs
  if convol_first:
    x=convolution(x)
    if batch_normalization:
      x = BatchNormalization()(x)
    if activation is not None:
      x = Activation(activation)(x)
  else:
    if batch_normalization:
      x = BatchNormalization()(x)
    if activation is not None:
      x = Activation(activation)(x)
    x = convolution(x)
  return x

In [ ]:
def resnet(input_shape, Depth, Num_of_Classes=10):
  Num_of_Filters = 16
  Num_of_Residual_Blocks = 3

  inputs = Input(shape=input_shape)
  x = single_resnet_layer(inputs=inputs)

  for stack in range(3):
    for Residual_Block in range(Num_of_Residual_Blocks):
      strides = 1
      if stack > 0 and Residual_Block == 0:
        strides = 2
      y = single_resnet_layer(inputs=x,Num_of_Filters=Num_of_Filters,strides=strides)
      y = single_resnet_layer(inputs=y,Num_of_Filters=Num_of_Filters,activation = None)
      if stack > 0 and Residual_Block == 0:
        x = single_resnet_layer(inputs=x,Num_of_Filters=Num_of_Filters,kernel_size=1,strides=strides,activation=None,batch_normalization=False)
      x = tensorflow.keras.layers.add([x,y])
      x = Activation('relu')(x)
    Num_of_Filters *=2
  
  x = AveragePooling2D(pool_size=8)(x)
  y = Flatten()(x)
  outputs = Dense(Num_of_Classes,activation='Softmax',kernel_initializer='he_normal')(y)
  model = Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
input_shape = x_training.shape[1:]

x_training = x_training.astype('float32')/255
x_testing = x_testing.astype('float32')/255
x_training_mean = np.mean(x_training, axis=0)
x_training -= x_training_mean
x_testing -= x_training_mean

y_training = tensorflow.keras.utils.to_categorical(y_training, Num_of_Classes)
y_testing = tensorflow.keras.utils.to_categorical(y_testing, Num_of_Classes)

model = resnet(input_shape=input_shape,Depth=Depth,Num_of_Classes=10)
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=Learning_Rate_Schedule(0)),metrics=['accuracy'])
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_147 (Conv2D)             (None, 32, 32, 16)   448         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_133 (BatchN (None, 32, 32, 16)   64          conv2d_147[0][0]                 
__________________________________________________________________________________________________
activation_133 (Activation)     (None, 32, 32, 16)   0           batch_normalization_133[0][0]    
____________________________________________________________________________________________

In [ ]:
def standard_time_format(secs):
  hours = int(secs/(60*60))
  minutes = int(secs%(60*60)/60)
  seconds = secs%60
  return f"{hours}:{minutes:>02}:{seconds:>05.2f}"

In [ ]:
import time
start_time = time.time()
lr_scheduler = LearningRateScheduler(Learning_Rate_Schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-06)
callbacks = [lr_reducer, lr_scheduler]

model.fit(x_training, y_training, batch_size = Batch_Size, epochs = Num_of_Iterations, callbacks=callbacks)

time_taken = time.time()-start_time
print("Time Taken: ".format(standard_time_format(time_taken)))

Epoch 1/200
1563/1563 [==============================] - 54s 14ms/step - loss: 1.4777 - accuracy: 0.5255
Epoch 2/200
1563/1563 [==============================] - 20s 13ms/step - loss: 1.0676 - accuracy: 0.6794
Epoch 3/200
1563/1563 [==============================] - 20s 13ms/step - loss: 0.9110 - accuracy: 0.7397
Epoch 4/200
1563/1563 [==============================] - 20s 13ms/step - loss: 0.8048 - accuracy: 0.7782
Epoch 5/200
1563/1563 [==============================] - 21s 13ms/step - loss: 0.7336 - accuracy: 0.8067
Epoch 6/200
1563/1563 [==============================] - 21s 14ms/step - loss: 0.6843 - accuracy: 0.8254
Epoch 7/200
1563/1563 [==============================] - 21s 14ms/step - loss: 0.6409 - accuracy: 0.8430
Epoch 8/200
1563/1563 [==============================] - 21s 14ms/step - loss: 0.6060 - accuracy: 0.8556
Epoch 9/200
1563/1563 [==============================] - 21s 14ms/step - loss: 0.5779 - accuracy: 0.8684
Epoch 10/200
1563/1563 [==============================]

In [ ]:
scores = model.evaluate(x_testing,y_testing)
print('Test loss: ',scores[0])
print('Test accuracy: ',scores[1])

313/313 [==============================] - 3s 7ms/step - loss: 1.0537 - accuracy: 0.8446
Test loss:  1.053669810295105
Test accuracy:  0.8446000218391418
